In [1]:
import pandas as pd
import os

In [2]:
s3_prefix = "s3://aegovan-data/pubmed_asbtract/predictions_largescale_ppimulticlass-bert-f1-2021-05-10-10_2021-07-01/"
s3_output_prefix = "{}_summary/".format(s3_prefix.rstrip("/"))
s3_data ="s3://aegovan-data/human_output/human_interactions_ppi_v2.json"

In [3]:
label_order = ["acetylation", "methylation", "phosphorylation", "dephosphorylation", "ubiquitination", "deubiquitination",  "other"]
pos_labels = list( filter(lambda x: x != 'other', label_order))
label_order_key = lambda x:  label_order.index(x)

label_title_map = {"other" : "Negative class"}

In [4]:
local_temp = "../temp"
local_temp_pred_dir = os.path.join( local_temp, "pred_results")
local_temp_wk_dir = os.path.join( local_temp, "wk")

In [ ]:
!rm -rf $local_temp
!mkdir -p $local_temp_pred_dir
!mkdir -p $local_temp_wk_dir

In [5]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os


def upload_file(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    input_tuples = ( (f,  s3_prefix) for f in glob.glob("{}/*".format(local_dir)))
    
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def get_s3file_size(bucket, key):
    s3 = boto3.client('s3')
    response = s3.head_object(Bucket=bucket, Key=key)
    size = response['ContentLength']
    return size
    
def download_files_min_files(s3_prefix, local_dir, min_file_size=310, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix) if get_s3file_size(s3_bucket, s3_key) > min_file_size )
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        


In [ ]:
%%time

download_files(s3_prefix, local_temp_pred_dir)

In [6]:
!ls -l $local_temp_dir | wc -l

      21


In [7]:
threshold_config = {'acetylation': {('confidence_std', 'count'): 5.0,
  ('confidence_std', 'mean'): 0.20221626758575403,
  ('confidence_std', 'std'): 0.010931891264366925,
  ('confidence_std', 'min'): 0.186287313699722,
  ('confidence_std', '25%'): 0.19900196790695102,
  ('confidence_std', '50%'): 0.20083150267601002,
  ('confidence_std', '75%'): 0.21166041493415802,
  ('confidence_std', 'max'): 0.21330013871192902,
  ('confidence', 'count'): 5.0,
  ('confidence', 'mean'): 0.5777932286262508,
  ('confidence', 'std'): 0.017217069729986746,
  ('confidence', 'min'): 0.555882334709167,
  ('confidence', '25%'): 0.572159707546234,
  ('confidence', '50%'): 0.57360166311264,
  ('confidence', '75%'): 0.5849224925041191,
  ('confidence', 'max'): 0.602399945259094},
 'dephosphorylation': {('confidence_std', 'count'): 29.0,
  ('confidence_std', 'mean'): 0.21133935400124204,
  ('confidence_std', 'std'): 0.07727310272430898,
  ('confidence_std', 'min'): 0.11373741179704601,
  ('confidence_std', '25%'): 0.141093701124191,
  ('confidence_std', '50%'): 0.18553803861141202,
  ('confidence_std', '75%'): 0.255215793848037,
  ('confidence_std', 'max'): 0.41554290056228604,
  ('confidence', 'count'): 29.0,
  ('confidence', 'mean'): 0.8063360175182077,
  ('confidence', 'std'): 0.13017405501056736,
  ('confidence', 'min'): 0.32787588238716103,
  ('confidence', '25%'): 0.7709274291992181,
  ('confidence', '50%'): 0.8457427024841301,
  ('confidence', '75%'): 0.8899683952331541,
  ('confidence', 'max'): 0.914224922657012},
 'deubiquitination': {('confidence_std', 'count'): 2.0,
  ('confidence_std', 'mean'): 0.1863851696252815,
  ('confidence_std', 'std'): 0.002637275825296506,
  ('confidence_std', 'min'): 0.184520334005355,
  ('confidence_std', '25%'): 0.18545275181531826,
  ('confidence_std', '50%'): 0.1863851696252815,
  ('confidence_std', '75%'): 0.18731758743524476,
  ('confidence_std', 'max'): 0.18825000524520802,
  ('confidence', 'count'): 2.0,
  ('confidence', 'mean'): 0.4542059451341625,
  ('confidence', 'std'): 0.010661192844799884,
  ('confidence', 'min'): 0.446667343378067,
  ('confidence', '25%'): 0.45043664425611474,
  ('confidence', '50%'): 0.4542059451341625,
  ('confidence', '75%'): 0.4579752460122103,
  ('confidence', 'max'): 0.461744546890258},
 'methylation': {('confidence_std', 'count'): 9.0,
  ('confidence_std', 'mean'): 0.20187029076947058,
  ('confidence_std', 'std'): 0.011129410572280824,
  ('confidence_std', 'min'): 0.185736715793609,
  ('confidence_std', '25%'): 0.19583970308303802,
  ('confidence_std', '50%'): 0.19923907518386802,
  ('confidence_std', '75%'): 0.210324048995971,
  ('confidence_std', 'max'): 0.21908366680145203,
  ('confidence', 'count'): 9.0,
  ('confidence', 'mean'): 0.7966000636418656,
  ('confidence', 'std'): 0.01647822043812186,
  ('confidence', 'min'): 0.7693868875503541,
  ('confidence', '25%'): 0.780769705772399,
  ('confidence', '50%'): 0.798503041267395,
  ('confidence', '75%'): 0.809625148773193,
  ('confidence', 'max'): 0.8169981241226191},
 'other': {('confidence_std', 'count'): 1116.0,
  ('confidence_std', 'mean'): 0.015799599189941234,
  ('confidence_std', 'std'): 0.0407698558038574,
  ('confidence_std', 'min'): 0.0008510624757030001,
  ('confidence_std', '25%'): 0.00117629769374575,
  ('confidence_std', '50%'): 0.0021780409151680004,
  ('confidence_std', '75%'): 0.007007123087532501,
  ('confidence_std', 'max'): 0.303181886672973,
  ('confidence', 'count'): 1116.0,
  ('confidence', 'mean'): 0.9903799230800303,
  ('confidence', 'std'): 0.026711376001797505,
  ('confidence', 'min'): 0.5133088827133171,
  ('confidence', '25%'): 0.9944566637277598,
  ('confidence', '50%'): 0.9974353015422815,
  ('confidence', '75%'): 0.998221039772033,
  ('confidence', 'max'): 0.9985265731811521},
 'phosphorylation': {('confidence_std', 'count'): 139.0,
  ('confidence_std', 'mean'): 0.09348429794416534,
  ('confidence_std', 'std'): 0.11442879568269237,
  ('confidence_std', 'min'): 0.006378921680152001,
  ('confidence_std', '25%'): 0.013172945939004001,
  ('confidence_std', '50%'): 0.034169171005487005,
  ('confidence_std', '75%'): 0.13673919439315751,
  ('confidence_std', 'max'): 0.469867438077926,
  ('confidence', 'count'): 139.0,
  ('confidence', 'mean'): 0.9306538756802781,
  ('confidence', 'std'): 0.09291076266425286,
  ('confidence', 'min'): 0.548133730888366,
  ('confidence', '25%'): 0.9297615289688106,
  ('confidence', '50%'): 0.9758448600769041,
  ('confidence', '75%'): 0.98560282588005,
  ('confidence', 'max'): 0.990909934043884},
 'ubiquitination': {('confidence_std', 'count'): 5.0,
  ('confidence_std', 'mean'): 0.1845212131738658,
  ('confidence_std', 'std'): 0.010037806334405529,
  ('confidence_std', 'min'): 0.174075484275817,
  ('confidence_std', '25%'): 0.177953422069549,
  ('confidence_std', '50%'): 0.18058878183364802,
  ('confidence_std', '75%'): 0.19217012822628002,
  ('confidence_std', 'max'): 0.197818249464035,
  ('confidence', 'count'): 5.0,
  ('confidence', 'mean'): 0.5571501374244686,
  ('confidence', 'std'): 0.07332355556921501,
  ('confidence', 'min'): 0.42922157049179005,
  ('confidence', '25%'): 0.5765218138694761,
  ('confidence', '50%'): 0.579930007457733,
  ('confidence', '75%'): 0.58320677280426,
  ('confidence', 'max'): 0.616870522499084}}

In [50]:
def get_summary_df(local_temp_pred_dir, use_std=True, conf_percentile="50%", confidence_std_percentile="50%"):
    list_df_high_quality = []
    list_df_summary = []
    for f in os.listdir(local_temp_pred_dir):
        df = pd.read_json(os.path.join(local_temp_pred_dir, f), orient="records" )
        

        list_df_summary.append(df[["prediction", "confidence", "confidence_std", "pubmedId", "participant1Id", "participant2Id", "participant1Name", "participant2Name"]])

        # Filter below threshold items
        high_quality_frames = []
        for k,t in threshold_config.items():
            conf_median = t[('confidence', conf_percentile)]
            conf_std_median = t[('confidence_std', confidence_std_percentile)]
            
            # HQ filter query
            qry = "prediction == '{}' and confidence >= {} and confidence_std <= {}" .format(k, conf_median, conf_std_median)
            if not use_std:
                qry = "prediction == '{}' and confidence >= {} " .format(k, conf_median)
           
            high_quality_frames.append(df.query(qry))

        high_quality_df = pd.concat(high_quality_frames)

        list_df_high_quality.append(high_quality_df)

    
    return pd.concat(list_df_high_quality), pd.concat(list_df_summary)



In [51]:
%%time 


df_high_quality, df_summary = get_summary_df (local_temp_pred_dir, use_std=True)

CPU times: user 2min 26s, sys: 7.09 s, total: 2min 34s
Wall time: 2min 35s


In [52]:
%%time

df_summary.groupby([ "participant1Id", "participant2Id", "prediction"])\
    .filter(lambda x: len(x)>1)\
    .groupby([ "participant1Id", "participant2Id", "prediction"])\
    .count()\
    .describe()

CPU times: user 24.8 s, sys: 319 ms, total: 25.2 s
Wall time: 25.2 s


,confidence,confidence_std,pubmedId,participant1Name,participant2Name
count,149405.000000,149405.000000,149405.000000,149405.000000,149405.000000
mean,8.051404,8.051404,8.051404,8.051404,8.051404
std,47.437537,47.437537,47.437537,47.437537,47.437537
min,2.000000,2.000000,2.000000,2.000000,2.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000
max,10229.000000,10229.000000,10229.000000,10229.000000,10229.000000


In [61]:
%%time

df_summary.groupby([ "participant1Id", "participant2Id", "prediction"])\
    .count()\
    .describe()

CPU times: user 1.91 s, sys: 83.6 ms, total: 2 s
Wall time: 2 s


,confidence,confidence_std,pubmedId,participant1Name,participant2Name
count,546588.000000,546588.000000,546588.000000,546588.000000,546588.000000
mean,2.927439,2.927439,2.927439,2.927439,2.927439
std,24.999565,24.999565,24.999565,24.999565,24.999565
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000
max,10229.000000,10229.000000,10229.000000,10229.000000,10229.000000


In [59]:
%%time

df_summary.groupby([ "participant1Id", "participant2Id", "prediction"])\
    .filter(lambda x: len(x)>3)\
    .sort_values(by = [ "participant1Id", "participant2Id", "prediction"])\
    .sample(n=10)

CPU times: user 23.7 s, sys: 367 ms, total: 24 s
Wall time: 24.1 s


,prediction,confidence,confidence_std,pubmedId,participant1Id,participant2Id,participant1Name,participant2Name
1022,phosphorylation,0.341951,0.342728,28191600,P01375,P06858,TNF-,Lipoprotein lipase
1472,phosphorylation,0.364973,0.369136,16484076,Q99622,Q9NZF1,C10,C15
582,phosphorylation,0.418808,0.442930,27186356,P01275,P43220,Glucagon-like peptide-1,GLP-1r
149,phosphorylation,0.405664,0.483579,2280610,P14784,P60568,IL-2 receptor,IL-2
267,phosphorylation,0.310521,0.320112,27588343,P05231,Q15848,interleukin-6,adiponectin
1665,phosphorylation,0.593183,0.445427,7734415,P01730,Q14624,CD4,gp120
1593,phosphorylation,0.543313,0.361742,17095601,O95999,Q9UDY8,Bcl10,Malt1
2573,phosphorylation,0.578782,0.387761,17395781,P00519,P07900,BCR-ABL,heat shock protein 90
754,phosphorylation,0.546119,0.461918,23769040,O14788,Q9Y6Q6,RANK ligand,receptor activator of nuclear factor-kappa B
503,ubiquitination,0.300279,0.219815,1743488,P07327,P14550,ADH,alcohol dehydrogenase


In [54]:
%%time

df_high_quality.groupby([ "participant1Id", "participant2Id", "prediction"])\
    .filter(lambda x: len(x)>1)\
    .groupby([ "participant1Id", "participant2Id", "prediction"])\
    .count()\
    .describe()

CPU times: user 247 ms, sys: 9.24 ms, total: 256 ms
Wall time: 254 ms


,pubmedId,participant1Name,participant2Name,abstract,normalised_abstract,annotations,gene_to_uniprot_map,normalised_abstract_annotations,other,phosphorylation,dephosphorylation,methylation,ubiquitination,acetylation,deubiquitination,confidence,confidence_std,raw_confidence
count,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000,541.000000
mean,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360,3.105360
std,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317,2.666317
min,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000


In [55]:
%%time

df_high_quality.groupby([ "participant1Id", "participant2Id", "prediction"])\
    .filter(lambda x: len(x)>1)\
    .groupby([  "prediction"])[  "prediction"]\
    .count()

CPU times: user 195 ms, sys: 1.3 ms, total: 196 ms
Wall time: 195 ms


prediction
dephosphorylation       4
methylation             4
phosphorylation      1672
Name: prediction, dtype: int64

In [73]:
%%time

df_high_quality.groupby([ "participant1Id", "participant2Id", "prediction"])\
    .filter(lambda x: len(x)>1 )\
    .sample(n=10)

CPU times: user 232 ms, sys: 329 ms, total: 561 ms
Wall time: 857 ms


,pubmedId,participant1Id,participant1Name,participant2Id,participant2Name,abstract,normalised_abstract,annotations,gene_to_uniprot_map,normalised_abstract_annotations,...,phosphorylation,dephosphorylation,methylation,ubiquitination,acetylation,deubiquitination,prediction,confidence,confidence_std,raw_confidence
1424,24999733,P12931,Src,Q05397,FAK,It is known that physico/chemical alterations ...,It is known that physico/chemical alterations ...,"[{'start': '334', 'end': '337', 'name': 'FAK',...","{'5747': 'Q05397', '6714': 'P12931'}","[{'charOffset': 334, 'len': 6, 'text': 'Q05397...",...,0.987649,0.001958,0.002220,0.001217,0.001394,0.001138,phosphorylation,0.987649,0.008336,"[0.9858871102333061, 0.973021805286407, 0.9881..."
1680,24360952,P12931,Src,Q05397,Focal adhesion kinase,Focal adhesion kinase (FAK) regulates cell adh...,"Q05397 (Q05397) regulates cell adhesion, migra...","[{'start': '0', 'end': '21', 'name': 'Focal ad...","{'5747': 'Q05397', '6714': 'P12931'}","[{'charOffset': 0, 'len': 6, 'text': 'Q05397'}...",...,0.988896,0.001967,0.002135,0.001252,0.001147,0.001262,phosphorylation,0.988896,0.008984,"[0.9881117343902581, 0.9700342416763301, 0.993..."
553,1532335,P06493,p34cdc2,P12004,cyclin,Phosphorylation of p34cdc2 can both positively...,Phosphorylation of P06493 can both positively ...,"[{'start': '19', 'end': '26', 'name': 'p34cdc2...","{'983': 'P06493', '5111': 'P12004'}","[{'charOffset': 19, 'len': 6, 'text': 'P06493'...",...,0.985637,0.003726,0.001752,0.001175,0.001129,0.001172,phosphorylation,0.985637,0.008471,"[0.9883902072906491, 0.9698903560638421, 0.988..."
382,25116692,P10914,interferon regulatory factor 1,P42224,STAT1,Signal transducer and activator of transcripti...,Signal transducer and activator of transcripti...,"[{'start': '134', 'end': '140', 'name': 'cattl...","{'6772': 'P42224', '3659': 'P10914', '1154': '...","[{'charOffset': 173, 'len': 6, 'text': 'P42224...",...,0.979223,0.003203,0.002287,0.003293,0.002100,0.002501,phosphorylation,0.979223,0.016470,"[0.9869648218154901, 0.974447607994079, 0.9919..."
1021,2530588,P06239,p56lck,P08575,CD45,CD45 is a family of high molecular weight leuk...,P08575 is a family of high molecular weight le...,"[{'start': '0', 'end': '4', 'name': 'CD45', 't...","{'5788': 'P08575', '995': 'P30307', '3932': 'P...","[{'charOffset': 0, 'len': 6, 'text': 'P08575'}...",...,0.979688,0.002188,0.003386,0.003284,0.002365,0.002786,phosphorylation,0.979688,0.025853,"[0.9846757054328911, 0.9845895171165461, 0.986..."
1971,19014654,P28482,Erk,P31749,Akt,The intracellular signalling mechanisms that r...,The intracellular signalling mechanisms that r...,"[{'start': '147', 'end': '150', 'name': 'Akt',...","{'207': 'P31749', '5594': 'P28482'}","[{'charOffset': 147, 'len': 6, 'text': 'P31749...",...,0.987237,0.001368,0.001577,0.001029,0.001181,0.001007,phosphorylation,0.987237,0.009956,"[0.9675111174583431, 0.9771018624305721, 0.993..."
2317,10938077,P68400,casein kinase II,Q04206,RelA,Nuclear factor kappaB (NF-kappaB)/Rel transcri...,Nuclear factor kappaB (P19838)/Q04864 transcri...,"[{'start': '23', 'end': '32', 'name': 'NF-kapp...","{'4790': 'P19838', '5966': 'Q04864', '7124': '...","[{'charOffset': 23, 'len': 6, 'text': 'P19838'...",...,0.987527,0.002463,0.001910,0.001363,0.001692,0.001164,phosphorylation,0.987527,0.011382,"[0.993252217769622, 0.964790046215057, 0.99266..."
936,19654415,P29474,endothelial nitric oxide synthase,P31749,Akt1,Akt1 is critical for many in vivo functions; h...,P31749 is critical for many in vivo functions;...,"[{'start': '0', 'end': '4', 'name': 'Akt1', 't...","{'207': 'P31749', '4846': 'P29474', '3091': 'Q...","[{'charOffset': 0, 'len': 6, 'text': 'P31749'}...",...,0.982303,0.002332,0.002695,0.001893,0.001746,0.001687,phosphorylation,0.982303,0.017464,"[0.951728582382202, 0.9657522439956661, 0.9918..."
2060,15996660,P01375,tumor necrosis factor,P47712,cPLA(2)alpha,Cytosolic phospholipase A(2)alpha (cPLA(2)alph...,Cytosolic phospholipase A(2)alpha (P4771

In [ ]:
import matplotlib.gridspec as gridspec

import numpy as np

def box_plot_prediction_confidence(df, df_high_quality, subplot_spec, title_prefix="", set_title=True):
    interaction_types = label_order
    
    num_plots = len(interaction_types)
    gs = gridspec.GridSpecFromSubplotSpec(1, len(label_order), subplot_spec=subplot_spec)
    
    for i, interaction in enumerate(interaction_types):
        df_interaction = df.query(f"prediction == '{interaction}'")
        df_interaction_high_quality = df_high_quality.query(f"prediction == '{interaction}'")
        
        if len(df_interaction) == 0: continue
        
        ax = fig.add_subplot(gs[0, i])
    
    
        # Rename columns
        new_column_names = {"confidence":"c", 
                           "confidence_std" : "v"
                            }
        df_interaction = df_interaction.rename(columns = new_column_names)
        df_interaction_high_quality=df_interaction_high_quality.rename(columns = new_column_names)
        
        
        # Style and formatting..
        
        if set_title:
            ax.set_title("{}{}\nT={} (HQ={}%)".format( title_prefix,
                                               label_title_map.get(interaction, interaction).title(),
                                               len(df_interaction),
                                               round(100 * len(df_interaction_high_quality)/len(df_interaction),2)                
                                              )
                        )
        ax.set_ylim(0,1)
        
        ax.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            labelbottom=False)
        
        ax.spines['bottom'].set_color('grey')
        ax.spines['top'].set_color('grey') 
        ax.spines['right'].set_color('grey')
        ax.spines['left'].set_color('grey')
        
        
        # Plot violin plot        
        ax.violinplot(df_interaction[["c", "v" ]],  showmeans=True )
        if len(df_interaction_high_quality) > 0:
            ax.violinplot(df_interaction_high_quality[["c", "v" ]],  showmeans=True )
        
        
        x_labels = ['C', 'V']
        
        ax.xaxis.set_tick_params(direction='out')
        ax.xaxis.set_ticks_position('bottom')
        ax.set_xticks(np.arange(1, len(x_labels) + 1))
        ax.set_xticklabels(x_labels)
        ax.set_xlim(0.25, len(x_labels) + 0.75)
        
        
       



        


In [ ]:
fig = plt.figure(figsize=(25 , 7))
gs = fig.add_gridspec(nrows=2)

box_plot_prediction_confidence(df_summary, df_high_quality, gs[0])

plt.savefig("largescaleprediction_distribution.pdf", bbox_inches="tight")

In [ ]:
df_summary.shape

In [ ]:
df_summary.groupby("prediction").describe().T

In [ ]:
df_high_quality.groupby(["prediction"])[["prediction", "confidence", "confidence_std"]].describe().T

In [ ]:
df_high_quality.shape

In [ ]:
download_file(s3_data, local_temp_wk_dir)

In [ ]:
data_file = os.path.join(local_temp_wk_dir, s3_data.split("/")[-1])
data_training_full_df = pd.read_json(data_file)

In [ ]:
data_training_full_df.shape

In [ ]:
data_training_full_df.head(n=2)

In [ ]:
data_training_full_df["participants"].sample(n=2).iloc[0]

In [ ]:
def get_partipants_key_raw(participants):
    """
    Example input
    [{'uniprotid': 'P19388',
  'alias': [['rpab1_human'],
   ['DNA-directed RNA polymerases I, II, and III subunit RPABC1'],
   ['POLR2E'],
   ['DNA-directed RNA polymerase II subunit E'],
   ['RPB5 homolog'],
   ['DNA-directed RNA polymerase II 23 kDa polypeptide'],
   ['XAP4']],
  'alternative_uniprots': ['Q6PIH5', 'Q9BT06', 'O43380', 'B2R6L4', 'D6W5Y1']},
 {'uniprotid': 'Q96SB4',
  'alias': [['srpk1_human'],
   ['SRSF protein kinase 1'],
   ['Serine/arginine-rich protein-specific kinase 1'],
   ['SFRS protein kinase 1'],
   ['SRPK1']],
  'alternative_uniprots': ['Q5R365', 'Q5R364', 'B4DS61', 'Q8IY12', 'Q12890']}]
    """
    participant_uniprot = []
    for p in participants:
        
        participant_uniprot.append(str(p["uniprotid"]))
        
        
    result = get_partipants_key(participant_uniprot)
    
    return result

def get_partipants_key(list_uniprot):
    participant_uniprot=sorted(filter (lambda x: x is not None, list_uniprot))
    
    result = "#".join(participant_uniprot)
    
    return result


def is_in_training(df, training_df):
    training_participants = training_df["participants"].apply(get_partipants_key_raw)
    df_particpiants = df[["participant2Id", "participant1Id"]].apply(lambda x : get_partipants_key([x["participant2Id"],x["participant2Id"]]), axis=1)
    return df_particpiants.isin(training_participants)

def is_in_training_pubmed(df, training_df):
    return df["pubmedId"].isin(training_df["pubmedId"])

In [ ]:
df_high_quality["PubmedInTrainingData"] = is_in_training_pubmed( df_high_quality, data_training_full_df)

In [ ]:
df_high_quality[~df_high_quality.PubmedInTrainingData].shape

In [ ]:
c_df=pd.DataFrame(df_summary.query("prediction != 'other'")\
                  .groupby([ "prediction"]).size()).rename(columns={0: "all_count"})
tmp_df = pd.DataFrame(df_high_quality[~df_high_quality.PubmedInTrainingData]\
      .groupby('prediction').size())\
      .rename(columns={0: "filter_count"})

tmp_df = tmp_df.merge(c_df, left_index=True,  right_index=True, how="right")\
      [[ "all_count", "filter_count"]]

print(tmp_df.to_latex( index=True))

tmp_df

In [ ]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [ ]:
df_high_quality.sample(n=10)

In [ ]:
df_high_quality["unique_gene_count"] = df_high_quality["gene_to_uniprot_map"].apply(lambda x: len(x))

In [ ]:
df_high_quality.groupby("prediction")[["confidence","unique_gene_count"]].describe()

In [ ]:
predictions_above_threshold_file = "predictions_above_threshold.json"
df_high_quality.to_json(predictions_above_threshold_file, orient='records')

In [ ]:
upload_file(predictions_above_threshold_file, "{}/".format(s3_output_prefix.rstrip("/")))

In [ ]:
samples_per_interaction = 30

samples_subset = df_high_quality.query("PubmedInTrainingData == False")\
                .groupby('prediction', group_keys=False)\
                .apply(lambda x: x.sample(min(len(x), samples_per_interaction),random_state=45))

samples_subset.groupby(["prediction"])["prediction"].count()

In [ ]:
samples_subset.columns

### Create ground truth training jobs

In [ ]:
import json
import json
def create_manifest_file(df, outfile):
    items = df.to_dict(orient='records' )
    with open(outfile , "w") as f:
        for item in items:
            # Write without new lines
            item_m  = {}
            item_m["source"] = json.dumps(item)
            f.write(json.dumps(item_m).replace("\n", "\t"))
            f.write("\n")

In [ ]:
def create_manifest_per_interaction(samples_subset_df, s3_output_prefix):
    samples_subset_file = "predictions_sample_subset.json"
    samples_subset_df.to_json(samples_subset_file, orient='records')
    upload_file(samples_subset_file, "{}/".format(s3_output_prefix.rstrip("/")))


    manifest_file = "predictions_sample_subset.mainfest"
    create_manifest_file(samples_subset_df, manifest_file)
    upload_file(manifest_file, "{}/".format(s3_output_prefix.rstrip("/")))

    # Create one manifest file per interaction type
    m_files=[]
    for i in list(samples_subset_df["prediction"].unique()):
        manifest_file = "predictions_sample_subset_{}.mainfest".format(i)
        s3_manifest_file = "{}/{}".format(s3_output_prefix.rstrip("/"), manifest_file)
        create_manifest_file(samples_subset_df.query("prediction == '{}'".format(i)), manifest_file)
        upload_file(manifest_file, s3_manifest_file)
        
        m_files.append(s3_manifest_file)
    return m_files


In [ ]:
s3_manifests = create_manifest_per_interaction(samples_subset, s3_output_prefix)

In [ ]:
import boto3
import sagemaker

In [ ]:
from datetime import datetime

def create_groundtruth_labelling_job(s3_manifest, s3_gt_output, s3_template, pre_lambda, post_lambda, role, workforce_name, job_name, label_attribute_name="prediction", workforce_type= "private-crowd" ):
    client = boto3.client('sagemaker')
    
    sagemaker_session = sagemaker.Session()
    account_id =  boto3.client('sts').get_caller_identity().get('Account')
    region = boto3.session.Session().region_name
    
    workforce_arn = "arn:aws:sagemaker:{}:{}:workteam/{}/{}".format(region, account_id, workforce_type, workforce_name)
    role_arn = "arn:aws:iam::{}:role/{}".format( account_id,  role)
    pre_lambda_arn = "arn:aws:lambda:{}:{}:function:{}".format(region, account_id,  pre_lambda)
    post_lambda_arn = "arn:aws:lambda:{}:{}:function:{}".format(region, account_id,  post_lambda)
    
    num_workers_per_object = 1
    task_time_limit_sec = 60  * 60 * 5
    task_availablity_sec =60  * 60 * 24 * 10
    
    job = client.create_labeling_job(LabelingJobName=job_name
                                    ,LabelAttributeName = label_attribute_name
                                    ,InputConfig = {
                                        "DataSource": {
                                            'S3DataSource': {
                                            'ManifestS3Uri': s3_manifest
                                            }
                                        }
                                        
                                    }
                                  ,OutputConfig={
                                        'S3OutputPath': s3_gt_output
                                    }

                                  , RoleArn = role_arn
                                  , HumanTaskConfig={
                                    'WorkteamArn': workforce_arn,
                                    'UiConfig': {
                                        'UiTemplateS3Uri': s3_template
                                    },
                                    'PreHumanTaskLambdaArn': pre_lambda_arn,
                                    'TaskKeywords': [
                                        'PPI',
                                    ],
                                    'TaskTitle': 'Verify PPI extraction for protein {}'.format(s3_manifest.split("/")[-1]),
                                    'TaskDescription': 'Verifies PPi extraction',
                                    'NumberOfHumanWorkersPerDataObject': num_workers_per_object,
                                    'TaskTimeLimitInSeconds': task_time_limit_sec,
                                    'TaskAvailabilityLifetimeInSeconds': task_availablity_sec,
                                    'MaxConcurrentTaskCount': 10,
                                    'AnnotationConsolidationConfig': {
                                        'AnnotationConsolidationLambdaArn': post_lambda_arn
                                    }
                                }
                            )
    
    return job
    
    

def create_groundtruth_labelling_multiple_jobs(lst_s3_manifests, s3_gt_output, s3_template, pre_lambda, post_lambda, role, workforce_name, job_prefix ="ppi", label_attribute_name="class"):
    job_prefix = "{}-{}".format(job_prefix , datetime.now().strftime("%Y%m%d%H%M%S"))
    for s3_manifest in lst_s3_manifests:
        job_name = "{}-{}".format( job_prefix, s3_manifest.split("/")[-1].split("_")[-1].split(".")[0])
        print(f"Creating job {job_name}")
        create_groundtruth_labelling_job(s3_manifest, s3_gt_output, s3_template, pre_lambda, post_lambda, role, workforce_name, job_name)

In [ ]:
import urllib.request

def download_template(template_url):
    with urllib.request.urlopen(template_url) as f:
        html = f.read().decode('utf-8')

    with open("template.html", "w") as f:
        f.write(html)
    
download_template('http://raw.githubusercontent.com/elangovana/ppi-sagemaker-groundtruth-verification/main/src/template/template.html')

In [67]:
role_name = "service-role/AmazonSageMaker-ExecutionRole-20210104T161547"
pre_lambda="Sagemaker-ppipreprocessing"
post_lambda="sagemaker-ppipostprocessing"
s3_gt_output = "{}/gt_output/".format(s3_prefix.rstrip("/"))
workforce_name = "ppi-team"
s3_template_file = "{}/template.html".format(s3_prefix.rstrip("/"))

In [ ]:


upload_file("template.html", s3_template_file )
create_groundtruth_labelling_multiple_jobs (s3_manifests,
                                            s3_gt_output, 
                                            s3_template_file,
                                            pre_lambda, 
                                            post_lambda, 
                                            role_name,
                                            workforce_name)

## Verify the ground truth jobs

In [68]:
ground_truth_jobs = [
"ppi-20210619153907-ubiquitination",
"ppi-20210619153907-phosphorylation",
"ppi-20210619153907-methylation",
"ppi-20210619153907-dephosphorylation",
"ppi-20210619153907-acetylation"
]

In [69]:
import pandas, json,   ast


def load_manifest_file(manifest_file):
    with open(manifest_file) as f:
        df_list = []
        for l in  f.readlines():
            record = json.loads(l)
            result = json.loads(record["source"])
            meta_key = list([k for k,_ in  record.items() if k.endswith( "-metadata" )])[0]
            result ["human_result"] = record[meta_key.replace("-metadata","")]["result"]
            
            df_list.append(result)
        return df_list
    
def load_manifests(manifest_files):
    l = []
    if isinstance (manifest_files, str): manifest_files=[manifest_files]
    for f in manifest_files:
        l.extend(load_manifest_file(f))
        
    return pd.DataFrame(l)
    
def load_manifests_s3(s3_manifest_files, local_dir):
    l = []
    if isinstance (s3_manifest_files, str): manifest_files=[s3_manifest_files]
    for s3_file in s3_manifest_files:
        manifest_file=os.path.join(local_dir , s3_file.split("/")[-1] )
        download_file(s3_file, local_dir)
        l.extend(load_manifest_file(manifest_file))
        
    return pd.DataFrame(l)

In [70]:
s3_gt_output_mainifests = ["{}/{}/manifests/output/output.manifest".format(s3_gt_output.rstrip("/"),j) for j in ground_truth_jobs]
df = load_manifests_s3(s3_gt_output_mainifests,local_temp)

In [71]:
df_gt_summary = df.groupby(["prediction", "human_result"])["human_result"].count().unstack().fillna(0).T
df_gt_summary["total"] = df_gt_summary.sum(axis=1)
df_gt_summary.loc["total"]= df_gt_summary.sum(axis=0)

In [72]:
df_gt_summary

prediction,acetylation,dephosphorylation,methylation,phosphorylation,ubiquitination,total
human_result,,,,,,
Correct,0.0,11.0,11.0,6.0,0.0,28.0
Incorrect - DNA Methylation,0.0,0.0,2.0,0.0,0.0,2.0
Incorrect - NER,0.0,2.0,1.0,3.0,0.0,6.0
Incorrect - No trigger word,0.0,1.0,0.0,2.0,4.0,7.0
Incorrect - Opposite type,0.0,1.0,0.0,0.0,0.0,1.0
Incorrect - relationship not described,0.0,14.0,4.0,19.0,0.0,37.0
Not - sure,1.0,0.0,1.0,0.0,0.0,2.0
total,1.0,29.0,19.0,30.0,4.0,83.0
